# Overview

In this Jupyter notebook, I demonstrate the process of training an object detection model for car detection using YOLOv5. The main goal is to fine-tune the pre-trained YOLOv5 model on a custom dataset of car images to improve its performance for detecting cars in images specific to the use case.

The notebook is organized into the following steps:

1. Preparation of the custom car dataset: Splitting the dataset into training and validation sets, and converting the annotations into a format suitable for use with YOLOv5.
2. Training YOLOv5 on the custom car dataset: Running the training script with the necessary parameters, such as image size, batch size, number of epochs, dataset configuration file, pre-trained weights, and the name of the training run.
3. Validating the trained YOLOv5 model: Running the validation script with the necessary parameters, such as dataset configuration file, trained weights, image size, IOU threshold, and whether to use half-precision inference.

By following these steps, I aim to achieve a better-performing car detection model fine-tuned for the specific use case.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
cd gdrive/MyDrive/Datrix

/content/gdrive/MyDrive/Datrix


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
cd yolov5

/content/gdrive/MyDrive/Datrix/yolov5


In [ ]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.3
    Uninstalling setuptools-63.4.3:
      Successfully uninstalled setuptools-63.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.


In [ ]:
import os
import json
import shutil
import cv2

In [ ]:
# Load the annotations file
with open('../Data/cars/annotations_sample.json') as json_file:
    annotations = json.load(json_file)

# Create the necessary directories
os.makedirs('../Data/car/train', exist_ok=True)
os.makedirs('../Data/car/val', exist_ok=True)
os.makedirs('../Data/car/labels/train', exist_ok=True)
os.makedirs('../Data/car/labels/val', exist_ok=True)

In [ ]:
# Split ratio for train and val datasets
split_ratio = 0.8
train_size = int(len(annotations['annotations']) * split_ratio)

In [ ]:
def convert_bbox_to_yolo(bbox, img_width, img_height):
    """
      Convert a bounding box from the format (x_min, y_min, width, height) to
      the YOLO format (x_center, y_center, width, height), where x_center and
      y_center are the center coordinates of the bounding box relative to the
      image dimensions, and width and height are the relative sizes of the
      bounding box with respect to the image dimensions.

      Args:
          bbox (tuple): A tuple containing the x_min, y_min, width, and height
                        of the bounding box in the format (x_min, y_min, width, height).
          img_width (int): The width of the image in pixels.
          img_height (int): The height of the image in pixels.

      Returns:
          list: A list containing the converted bounding box in YOLO format
                [x_center, y_center, width, height], where all values are
                relative to the image dimensions.
    """ 
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    width /= img_width
    height /= img_height
    return [x_center, y_center, width, height]

In [ ]:
for i, annotation in enumerate(annotations['annotations']):

    """
      Iterate through all annotations and split them into train and validation sets.
      For each annotation, create the corresponding YOLO-format label files and copy
      the image files to the appropriate train or validation folders.
    """
    file_name = annotation['file_name']
    image_path = os.path.join('../Data/cars', file_name)
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    if i < train_size:
        # Copy the image to the train folder
        shutil.copy(image_path, os.path.join('../Data/car/train', file_name))

        # Write the corresponding label file
        with open(os.path.join('../Data/car/labels/train', file_name[:-4] + '.txt'), 'w') as label_file:
            """
              Convert the bounding box to YOLO format and write it to the label
              file for the training set.
            """
            yolo_bbox = convert_bbox_to_yolo(annotation['bbox'], img_width, img_height)
            label_file.write(f"0 {' '.join(map(str, yolo_bbox))}\n")

    else:
        # Copy the image to the val folder
        shutil.copy(image_path, os.path.join('../Data/car/val', file_name))

        # Write the corresponding label file
        with open(os.path.join('../Data/car/labels/val', file_name[:-4] + '.txt'), 'w') as label_file:
            """
              Convert the bounding box to YOLO format and write it to the label
              file for the validation set.
            """
            yolo_bbox = convert_bbox_to_yolo(annotation['bbox'], img_width, img_height)
            label_file.write(f"0 {' '.join(map(str, yolo_bbox))}\n")

In [ ]:
yaml_content = """
train: ../Data/car/train
val: ../Data/car/val

nc: 1
names: ['car']
"""

with open("car_dataset.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

In [ ]:
ls

benchmarks.py     data/       LICENSE          requirements.txt  tutorial.ipynb
car_dataset.yaml  Data/       models/          runs/             utils/
CITATION.cff      detect.py   __pycache__/     segment/          val.py
classify/         export.py   README.md        setup.cfg         yolov5s.pt
CONTRIBUTING.md   hubconf.py  README.zh-CN.md  train.py


## Training YOLOv5 on the Custom Car Dataset

Now that I have prepared the custom car dataset in the required format, I can train YOLOv5 using this dataset. To do this, I will run the `train.py` script, providing the necessary arguments such as image size, batch size, number of epochs, dataset configuration file, pre-trained weights, and the name of the training run.


**Note:** During the training process, I encountered an error like this:


This error indicates that the training script could not find any labels for the images in the training dataset. There might be several reasons for this issue, such as an incorrect path to the labels, improperly formatted label files, or missing label files. To resolve this issue, I double-checked the paths to the labels in the dataset configuration file and ensured that the label files were correctly formatted and located in the appropriate directories. For more information, consult the [YOLOv5 documentation on training custom data](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data).


In [ ]:
# Execute the YOLOv5 training script with the following parameters:
#   - Resize input images to 640x640 pixels (--img 640)
#   - Set the batch size to 16 (--batch 16)
#   - Train the model for 100 epochs (--epochs 100)
#   - Use the configuration file 'car_dataset.yaml' to specify dataset and training parameters (--data car_dataset.yaml)
#   - Initialize the model with pre-trained weights from 'yolov5s.pt' (--weights yolov5s.pt)
#   - Name the output model and related files with the prefix 'car_detector' (--name car_detector)

!python train.py --img 640 --batch 16 --epochs 100 --data car_dataset.yaml --weights yolov5s.pt --name car_detector

train: weights=yolov5s.pt, cfg=, data=car_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=car_detector, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0

## Validating the Trained YOLOv5 Model

After training the YOLOv5 model on the custom car dataset, I will validate its performance using the validation dataset. To do this, I will run the `val.py` script, providing the necessary arguments such as dataset configuration file, trained weights, image size, IOU threshold, and whether to use half-precision inference.


In [ ]:
# Execute the YOLOv5 validation script with the following parameters:
#   - Use the configuration file 'car_dataset.yaml' to specify dataset and validation parameters (--data car_dataset.yaml)
#   - Load the model weights from 'runs/train/car_detector/weights/best.pt' (--weights runs/train/car_detector/weights/best.pt)
#   - Resize input images to 640x640 pixels (--img 640)
#   - Set the Intersection over Union (IoU) threshold to 0.65 (--iou 0.65)
#   - Use FP16 half-precision mode for faster inference and lower memory usage (--half)

!python val.py --data car_dataset.yaml --weights runs/train/car_detector/weights/best.pt --img 640 --iou 0.65 --half

val: data=car_dataset.yaml, weights=['runs/train/car_detector/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CPU

Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Datrix/yolov5/val.py", line 409, in <module>
    main(opt)
  File "/content/gdrive/MyDrive/Datrix/yolov5/val.py", line 380, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/content/gdrive/MyDrive/Datrix/yolov5/val.py", line 143, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/content/gdrive/MyDrive/Datrix/yolov5/models